In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./data_ab.csv")
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2025-01-21 22:11:48.556739,control,old_page,0
1,804228,2025-01-12 08:01:45.159739,control,old_page,0
2,661590,2025-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2025-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2025-01-21 01:52:26.210827,control,old_page,1


In [3]:
df.shape

(294478, 5)

1. проверим дубликаты юзеров

In [4]:
len(df) == df["user_id"].nunique()

False

In [5]:
dupes = df["user_id"].value_counts()
dupes[dupes > 1]

user_id
805339    2
754884    2
722274    2
783176    2
898232    2
         ..
847058    2
889392    2
664150    2
835401    2
736955    2
Name: count, Length: 3894, dtype: int64

In [6]:
# 1. Находим user_id, которые встречаются больше 1 раза
dupes_ids = df["user_id"].value_counts()
dupes_ids = dupes_ids[dupes_ids > 1].index

# 2. Выводим все строки, где user_id дублируется
duplicate_rows = df[df["user_id"].isin(dupes_ids)]

# 3. Сортировка по user_id
duplicate_rows_sorted = duplicate_rows.sort_values(by="user_id")

duplicate_rows_sorted

,user_id,timestamp,group,landing_page,converted
230259,630052,2025-01-17 01:16:05.208766,treatment,new_page,0
213114,630052,2025-01-07 12:25:54.089486,treatment,old_page,1
22513,630126,2025-01-14 13:35:54.778695,treatment,old_page,0
251762,630126,2025-01-19 17:16:00.280440,treatment,new_page,0
183371,630137,2025-01-20 02:08:49.893878,control,old_page,0
...,...,...,...,...,...
142354,945703,2025-01-08 19:40:51.169351,control,new_page,0
186960,945797,2025-01-13 17:23:21.750962,control,old_page,0
40370,945797,2025-01-11 03:04:49.433736,control,new_page,1
165143,945971,2025-01-16 10:09:18.383183,control,old_page,0


2. удаляем несовпадения

In [9]:
df_clean = df[
    ((df["group"] == "treatment") & (df["landing_page"] == "new_page")) |
    ((df["group"] == "control") & (df["landing_page"] == "old_page"))
]
df_clean

,user_id,timestamp,group,landing_page,converted
0,851104,2025-01-21 22:11:48.556739,control,old_page,0
1,804228,2025-01-12 08:01:45.159739,control,old_page,0
2,661590,2025-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2025-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2025-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
294473,751197,2025-01-03 22:28:38.630509,control,old_page,0
294474,945152,2025-01-12 00:51:57.078372,control,old_page,0
294475,734608,2025-01-22 11:45:03.439544,control,old_page,0
294476,697314,2025-01-15 01:20:28.957438,control,old_page,0


In [10]:
len(df) - len(df_clean)

3893

In [11]:
dupes = df_clean["user_id"].value_counts()
dupes[dupes > 1]

user_id
773192    2
Name: count, dtype: int64

In [12]:
df_clean[df_clean["user_id"]==773192]

,user_id,timestamp,group,landing_page,converted
1899,773192,2025-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2025-01-14 02:55:59.590927,treatment,new_page,0


оставим первый визит пользователя

In [13]:
df_final = df_clean.sort_values("timestamp").drop_duplicates(subset="user_id", keep="first")
df_final.shape

(290584, 5)

In [14]:
df = df_final

3. делим на 2 датасета один - df_control, второй - df_treatment

In [15]:
df_control = df[df["group"]=="control"]
df_control.head()

,user_id,timestamp,group,landing_page,converted
184884,781507,2025-01-02 13:42:15.234051,control,old_page,0
83878,737319,2025-01-02 13:42:21.786186,control,old_page,0
112584,799109,2025-01-02 13:42:45.219901,control,old_page,0
126997,643921,2025-01-02 13:43:03.767329,control,old_page,1
189698,677659,2025-01-02 13:43:32.744957,control,old_page,0


In [16]:
df_treatment = df[df["group"]=="treatment"]
df_treatment.head()

,user_id,timestamp,group,landing_page,converted
131228,922696,2025-01-02 13:42:05.378582,treatment,new_page,0
102717,818377,2025-01-02 13:42:26.640581,treatment,new_page,0
158789,725857,2025-01-02 13:42:27.851110,treatment,new_page,0
191378,762651,2025-01-02 13:42:28.522322,treatment,new_page,0
111830,722516,2025-01-02 13:42:34.033708,treatment,new_page,0


In [17]:
df_control.shape

(145274, 5)

In [18]:
df_treatment.shape

(145310, 5)

In [19]:
n_control = len(df_control)
n_treatment = len(df_treatment)
total = n_control + n_treatment

print("Control %:", n_control/total * 100)
print("Treatment %:", n_treatment/total * 100)

Control %: 49.99380557773312
Treatment %: 50.00619442226688


Небольшая разница в размере групп — норма.
При случайном распределении и последующей очистке данных группы почти никогда
не получаются равными.
Главное, что перекос минимальный — значит сплит корректен.

# 2.проверка сплита

In [20]:
import pandas as pd
from scipy.stats import chi2_contingency

# выделяем день
df["date"] = pd.to_datetime(df["timestamp"]).dt.date

# считаем количество пользователей по дням и группам
daily_counts = (
    df.groupby(["date", "group"])["user_id"]
    .nunique()
    .unstack(fill_value=0)
)

# применяем χ²-тест
chi2, p, dof, expected = chi2_contingency(daily_counts)

print("Chi-square:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)

Chi-square: 13.420603994207953
p-value: 0.9207631079105322
Degrees of freedom: 22


В нашем эксперименте проведение A/A-теста не требуется, так как сплит формируется не вручную, а заранее встроен в продуктовую логику: пользователи получают либо old_page (control), либо new_page (treatment) на уровне системы. В отличие от искусственных A/A-тестов, здесь невозможно выдать обоим вариантам идентичную страницу для проверки, поскольку вариант интерфейса жёстко привязан к группе. Вместо этого корректность рандомизации подтверждается непосредственно структурой данных: группы имеют одинаковый порядок величин, распределение трафика по дням статистически не отличается (χ²-тест), динамика дневной конверсии стабильна и не показывает систематического смещения, а начало и окончание эксперимента происходят синхронно для обеих групп. Все ключевые свойства качественного сплита выполнены, поэтому отдельный A/A-тест в данном датасете не только избыточен, но и методологически невозможен из-за природы экспериментальных условий.